In [1]:
# ИУ5-65Б СОРОКИН АРТЕМ Вариант 9




from sklearn.datasets import load_iris 
import pandas as pd
import numpy as np
import random

from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV



iris = load_iris(return_X_y=False, as_frame=False)

In [2]:
print(type(iris))

<class 'sklearn.utils._bunch.Bunch'>


In [3]:
# data.head(10)

data = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

In [4]:
print(data)

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                  5.1               3.5                1.4               0.2   
1                  4.9               3.0                1.4               0.2   
2                  4.7               3.2                1.3               0.2   
3                  4.6               3.1                1.5               0.2   
4                  5.0               3.6                1.4               0.2   
..                 ...               ...                ...               ...   
145                6.7               3.0                5.2               2.3   
146                6.3               2.5                5.0               1.9   
147                6.5               3.0                5.2               2.0   
148                6.2               3.4                5.4               2.3   
149                5.9               3.0                5.1               1.8   

     target  
0       0.0  

In [5]:
# СОЗДАНИE ПРОПУСКОВ


# ЧИСЛОВЫХ ПРИЗНАКОВ

for i in range(8):
    k = random.randint(1, 150)
    data['petal length (cm)'][k] = np.nan
    
# КАТЕГОРИАЛЬНЫЙ ПРИЗНАК    
for i in range(8):
    k = random.randint(1, 150)
    data['target'][k] = np.nan
    
    

In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(data)

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                  5.1               3.5                1.4               0.2   
1                  4.9               3.0                1.4               0.2   
2                  4.7               3.2                1.3               0.2   
3                  4.6               3.1                1.5               0.2   
4                  5.0               3.6                1.4               0.2   
5                  5.4               3.9                1.7               0.4   
6                  4.6               3.4                1.4               0.3   
7                  5.0               3.4                1.5               0.2   
8                  4.4               2.9                1.4               0.2   
9                  4.9               3.1                1.5               0.1   
10                 5.4               3.7                1.5               0.2   
11                 4.8      

### ПРОПУСКИ ДАННЫХ

In [7]:
# Скопируем DataFrame чтобы попробовать несколько вариантов заполнения пропусков
# В данном случае, если проанализировать пропуски, становится понятно, что удобнее всего способ замены на значение 
# из следующей записи или из предыдущей. Так как записи в наборе данных более-менее отсортированные



# Попробуем сначала заполнить средним значением столбца (категориальный признак так заполнять не разумно)


df = data.copy()

df_test = df.fillna(df.mean())['petal length (cm)']

print (df.isnull().sum())

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_test)
    
    
# Среднее значение 3.733099. Выпадает из списка и может нарушить обучение модели в строках 27, 104, 122

sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    8
petal width (cm)     0
target               7
dtype: int64
0      1.400000
1      1.400000
2      1.300000
3      1.500000
4      1.400000
5      1.700000
6      1.400000
7      1.500000
8      1.400000
9      1.500000
10     1.500000
11     1.600000
12     1.400000
13     1.100000
14     1.200000
15     1.500000
16     1.300000
17     1.400000
18     1.700000
19     1.500000
20     1.700000
21     1.500000
22     1.000000
23     1.700000
24     1.900000
25     1.600000
26     1.600000
27     1.500000
28     3.759859
29     1.600000
30     1.600000
31     1.500000
32     1.500000
33     1.400000
34     1.500000
35     1.200000
36     1.300000
37     1.400000
38     1.300000
39     3.759859
40     1.300000
41     1.300000
42     1.300000
43     1.600000
44     1.900000
45     1.400000
46     1.600000
47     1.400000
48     1.500000
49     1.400000
50     4.700000
51     4.500000
52     4.900000
53     4.000000
54     4

### Заполнение пропусков значением из предыдущей записи
### Крайне удачный вариант, так как категориальные признаки в данном случае отсортированы

In [8]:
df_test = df.fillna(method='bfill')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_test)
    
# Получили практически исходный набор данных

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                  5.1               3.5                1.4               0.2   
1                  4.9               3.0                1.4               0.2   
2                  4.7               3.2                1.3               0.2   
3                  4.6               3.1                1.5               0.2   
4                  5.0               3.6                1.4               0.2   
5                  5.4               3.9                1.7               0.4   
6                  4.6               3.4                1.4               0.3   
7                  5.0               3.4                1.5               0.2   
8                  4.4               2.9                1.4               0.2   
9                  4.9               3.1                1.5               0.1   
10                 5.4               3.7                1.5               0.2   
11                 4.8      

### Перейдем к обучению модели

In [9]:
iris.data.shape 

(150, 4)

In [10]:
x = iris.data
y = iris.target

### Делим данные на тестовую выборку и тренировочную

In [11]:
#Делим данные на тренировочное и тестовое множество:

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
print(X_train.shape, X_test.shape, Y_test.shape, Y_train.shape)

(120, 4) (30, 4) (30,) (120,)


### Будем использовать метод ближайших соседей, так как из данных видно, что классы имеют четко различимые границы

In [12]:
K_range = range(1, 11)
scores = {}
scores_list = []
for k in K_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, Y_train)
    Y_pred = knn.predict(X_test)
    scores[k] = metrics.accuracy_score(Y_test, Y_pred)
    scores_list.append(metrics.accuracy_score(Y_test, Y_pred))
    
print (scores)

{1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 0.9666666666666667, 8: 1.0, 9: 1.0, 10: 1.0}


### Посмотрим какой подберет гиперпараметр встроенный метод KNeighborsClassifier

In [13]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x, y)

KNeighborsClassifier(n_neighbors=3)

In [14]:
classes = {0:'setosa', 1:'versicolor', 2:'virginica'}

In [15]:
X_new = [[3,4,5,2],
        [5,4,2,2,]]

In [16]:
Y_pred = knn.predict(X_new)

In [17]:
print (classes [Y_pred[0]])
print (classes [Y_pred[1]])

versicolor
setosa


In [18]:
#Создаем классификатор:

best_model = KNeighborsClassifier(
   n_neighbors=10, 
   weights='distance',
   algorithm='auto',
   leaf_size=30,
   metric='euclidean',
   metric_params=None,
   n_jobs=4
)

best_model.fit(X_train, Y_train)
predicted = best_model.predict(X_test)

print('Evaluation:\n', metrics.classification_report(Y_test, predicted))

Evaluation:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



### Видим, что модель может классифицировать данные, причем как показывает со 100% точностью. Дело в том, что числовые признаки сильно отличаются друг от друга при смене класса - это видно из набора данных. В таких случаях метод K-ближайших соседей показывает наибольшую точность прогнозирования.